# Low pass filter design

In [80]:
import scipy as sp
import numpy as np
from scipy.signal import kaiserord, lfilter, firwin, freqz,group_delay
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib widget

from pathlib import Path
import pickle

## Filters definition 


In [90]:
## Use firwin with a Kaiser window to create a lowpass FIR filter.
taps_lpf_1600_400 = firwin(numtaps=80, cutoff=200 ,fs=1600)
taps_lpf_400_200 = firwin(numtaps=64, cutoff=100 ,fs=400)
taps_lpf_200_100 = firwin(numtaps=64, cutoff=50 ,fs=200)
taps_lpf_100_50 = firwin(numtaps=64, cutoff=25 ,fs=100)
taps_lpf = firwin(30,cutoff=50,window='hamming',fs=400)

### Filter response

In [91]:
#------------------------------------------------
# Plot the magnitude response of the filter.
#------------------------------------------------
fs = 1600
taps = taps_lpf_1600_400
nyq_rate=fs/2.0
#
f,(ax1,ax2,ax3) = plt.subplots(nrows=3,sharex=True)
w, h = freqz(taps, worN=8000)
ax1.plot((w/np.pi)*nyq_rate,20*np.log10(np.absolute(h)) , linewidth=2)
ax2.plot((w/np.pi)*nyq_rate,np.absolute(h) , linewidth=2)
ax2.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Gain(dB)')
ax2.set_ylabel('Gain')
ax2.set_ylim(0.5, 1.005)
ax3.plot((w/np.pi)*nyq_rate,np.unwrap(np.angle(h)) , linewidth=2)
ax3.set_ylabel('Phase(angle)')



FigureCanvasNbAgg()

Text(0, 0.5, 'Phase(angle)')

### Header file generation

In [92]:
f2IQ31= lambda x: int(x*(2**31-1))
f2IQ31=np.vectorize(f2IQ31)

def print_FIR_coef(x, name ,alternating=False,n=4):
    s="#define\tLPF_{}_ORDER\t {}\n".format(name,len(x)-1)
    s+="#define LPF_{}_COEF\t{{\\\n".format(name)
    if alternating:
        y=x[::-1]
    
    for i,t in enumerate(y):
        if i>0:
            if (i+1)%n==0:
                s+=",\\\n".format(t)
            else:
                s+=", ".format(t)
        s+= str(t)
    if s[-1]!="\n":
        s+="}"
        
    return s

In [93]:
header_file_temp="""#ifndef FIR_COEFS_H_ 
#define FIR_COEFS_H_

/*
Filter coefficients of the low pass filters oof the trave application 
This file is generated using the trave_lpf_filter_design.ipynb ipython notebook
*/

//decimations factors
#define LPF_1600_TO_400_DECIMATION_FACTOR 4
#define LPF__DECIMATION_FACTOR 4  

//coefficients
{}
//------------
{}

#endif /* FIR_COEFS_H_ */
"""
columns=12
header_file_s=header_file_temp.format(
    print_FIR_coef(f2IQ31(taps_lpf_1600_400),"1600_TO_400",alternating=True,n=columns),
    print_FIR_coef(f2IQ31(taps_lpf),"",alternating=True,n=columns),

)

In [94]:
#print(header_file_s)

In [95]:
with  Path("").parent.joinpath("lpf_filters_coefs.h").open("w+") as f:
    f.write(header_file_s)

In [96]:
%ls

formato_lcd.ods      test_lpf_filters.ipynb         traveSM.h
lpf_filters_coefs.h  trave_lpf_filter_design.ipynb
